In [42]:
# importing required libraries
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D , Dense , Dropout , Flatten
import numpy as np 
from sklearn.model_selection import train_test_split, StratifiedKFold 
import matplotlib.pyplot as plt 
import cv2 
import pandas as pd 

In [20]:
!wget https://www.gti.ssr.upm.es/images/Data/Downloads/HandGestureDatabase/set_2.rar

--2021-09-27 02:02:18--  https://www.gti.ssr.upm.es/images/Data/Downloads/HandGestureDatabase/set_2.rar
Resolving www.gti.ssr.upm.es (www.gti.ssr.upm.es)... 138.4.32.2
Connecting to www.gti.ssr.upm.es (www.gti.ssr.upm.es)|138.4.32.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33102454881 (31G) [application/rar]
Saving to: ‘set_2.rar.2’

set_2.rar.2           7%[>                   ]   2.44G  8.17MB/s    in 3m 37s  


Cannot write to ‘set_2.rar.2’ (Success).


In [19]:
!unrar x "/content/set_2.rar"


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/set_2.rar


Would you like to replace the existing file set_2/test/subject_1/ground_truth/LabelingSession.mat
12100020 bytes, modified on 2014-08-07 13:55
with a new one
12100020 bytes, modified on 2014-08-07 13:55

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit 
User break

User break


In [39]:
# defining image folders
base_subfolder          =  '/content/set_2/training/'
activation_subfolder    =  base_folder + 'g01/subject_1/sequence'
cursor_subfolder        =  base_folder + 'g02/subject_1/sequence'
left_click_subfolder    =  base_folder + 'g03/subject_1/sequence'
right_click_subfolder   =  base_folder + 'g04/subject_1/sequence'
deactivation_subfolder  =  base_folder + 'g05/subject_1/sequence'

In [40]:
from glob import glob 

# finding filenames in respective subfolders
activation_filenames = glob(activation_subfolder + '/*')
cursor_filenames = glob(cursor_subfolder + '/*')
left_click_filenames = glob(left_click_subfolder + '/*')
right_click_filenames = glob(right_click_subfolder + '/*')
deactivation_filenames = glob(deactivation_subfolder + '/*')

# printing filename info 
print("Total {} images found in {} folder".format(len(activation), activation_subfolder))
print("Total {} images found in {} folder".format(len(cursor), cursor_subfolder))
print("Total {} images found in {} folder".format(len(left_click), left_click_subfolder))
print("Total {} images found in {} folder".format(len(right_click), right_click_subfolder))
print("Total {} images found in {} folder".format(len(deactivation), deactivation_subfolder))

Total 38 images found in /content/set_2/training/g01/subject_1/sequence folder
Total 38 images found in /content/set_2/training/g02/subject_1/sequence folder
Total 38 images found in /content/set_2/training/g03/subject_1/sequence folder
Total 38 images found in /content/set_2/training/g04/subject_1/sequence folder
Total 38 images found in /content/set_2/training/g05/subject_1/sequence folder


In [43]:
# creating a dataframe for containing the filepaths (complete absolute filepaths) and corresponding class labels
label_names   = ['activation','cursor','left_click','right_click','deactivation']
all_filepaths = activation_filenames + cursor_filenames + left_click_filenames + right_click_filenames + deactivation_filenames
all_labels    = [label_names[0]]*len(activation_filenames) + [label_names[1]]*len(cursor_filenames) + [label_names[2]]*len(left_click_filenames) + [label_names[3]]*len(right_click_filenames) + [label_names[4]]*len(deactivation_filenames)   
df = pd.DataFrame({'filename':all_filepaths, 'class':all_labels})
print(df.head())
print(df.describe())

                                            filename       class
0  /content/set_2/training/g01/subject_1/sequence...  activation
1  /content/set_2/training/g01/subject_1/sequence...  activation
2  /content/set_2/training/g01/subject_1/sequence...  activation
3  /content/set_2/training/g01/subject_1/sequence...  activation
4  /content/set_2/training/g01/subject_1/sequence...  activation
                                                 filename       class
count                                                3853        3853
unique                                               3853           5
top     /content/set_2/training/g02/subject_1/sequence...  left_click
freq                                                    1        1041


In [44]:
# performing train, test set splits
train_val_df, test_df = train_test_split(df, test_size=0.1, random_state=0, shuffle=True, stratify=df['class'])
print("Training+Validation set")
print(train_val_df.head())
print(train_val_df.describe())
print('*'*50)
print("Testing set")
print(test_df.head())
print(test_df.describe())
test_df['filename']

Training+Validation set
                                               filename         class
306   /content/set_2/training/g01/subject_1/sequence...    activation
441   /content/set_2/training/g01/subject_1/sequence...    activation
3757  /content/set_2/training/g05/subject_1/sequence...  deactivation
1046  /content/set_2/training/g02/subject_1/sequence...        cursor
2497  /content/set_2/training/g04/subject_1/sequence...   right_click
                                                 filename       class
count                                                3467        3467
unique                                               3467           5
top     /content/set_2/training/g03/subject_1/sequence...  left_click
freq                                                    1         937
**************************************************
Testing set
                                               filename         class
147   /content/set_2/training/g01/subject_1/sequence...    activation
126

147     /content/set_2/training/g01/subject_1/sequence...
1260    /content/set_2/training/g03/subject_1/sequence...
2875    /content/set_2/training/g04/subject_1/sequence...
3437    /content/set_2/training/g05/subject_1/sequence...
291     /content/set_2/training/g01/subject_1/sequence...
                              ...                        
697     /content/set_2/training/g01/subject_1/sequence...
3582    /content/set_2/training/g05/subject_1/sequence...
599     /content/set_2/training/g01/subject_1/sequence...
1084    /content/set_2/training/g02/subject_1/sequence...
2040    /content/set_2/training/g03/subject_1/sequence...
Name: filename, Length: 386, dtype: object